In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from jsonschema import validate, ValidationError

LLM_BASE_URL="https://aimodels.jadeglobal.com:8082/ollama/api"
LLM_MODEL="llama3.1:8b"
LLM_VERIFY_SSL=False
LLM_TIMEOUT=120

print("Script started...", flush=True)

# ==============================
# Load Environment Variables
# ==============================


print(f"Loaded BASE URL: {LLM_BASE_URL}", flush=True)
print(f"Using MODEL: {LLM_MODEL}", flush=True)


# ==============================
# API Connectivity Check
# ==============================
def check_api_connection():
    try:
        print("Checking API connectivity...", flush=True)
        response = requests.get(
            LLM_BASE_URL,
            verify=LLM_VERIFY_SSL,
            timeout=5
        )
        print("API is reachable.", flush=True)
        return True
    except requests.exceptions.RequestException:
        print("❌ API NOT RESPONDING", flush=True)
        return False


# ==============================
# Prompt Builder
# ==============================
def build_prompt(article: str) -> str:
    return f"""
Return ONLY valid JSON:

{{
  "summary": "string (max 150 words)",
  "important_points": ["string"],
  "key_themes": ["string"],
  "target_audience": "string"
}}

Article:
\"\"\"
{article}
\"\"\"
"""


# ==============================
# LLM API Call
# ==============================
def call_llm(prompt: str) -> str:
    url = f"{LLM_BASE_URL}/generate"

    payload = {
        "model": LLM_MODEL,
        "prompt": prompt,
        "stream": False
    }

    print("Calling LLM API...", flush=True)

    try:
        response = requests.post(
            url,
            json=payload,
            verify=LLM_VERIFY_SSL,
            timeout=LLM_TIMEOUT
        )

        response.raise_for_status()
        print("API responded successfully.", flush=True)

        return response.json().get("response", "")

    except requests.exceptions.Timeout:
        print("❌ API TIMEOUT - Not Responding", flush=True)
        raise Exception("API Timeout")

    except requests.exceptions.ConnectionError:
        print("❌ CONNECTION ERROR - API NOT RESPONDING", flush=True)
        raise Exception("Connection Error")

    except requests.exceptions.RequestException as e:
        print("❌ API ERROR:", str(e), flush=True)
        raise


# ==============================
# Main
# ==============================
if __name__ == "__main__":

    print("Starting program...\n", flush=True)

    # Check API first
    if not check_api_connection():
        print("\nStopping execution because API is not reachable.", flush=True)
        exit()

    sample_article = """
    Artificial intelligence is transforming industries by automating tasks,
    improving decision-making, and enabling predictive analytics.
    """

    try:
        prompt = build_prompt(sample_article)
        result = call_llm(prompt)

        print("\nRaw LLM Output:\n", flush=True)
        print(result, flush=True)

    except Exception as e:
        print("\nExecution failed:", str(e), flush=True)

    print("\nProgram finished.", flush=True)

Script started...
Loaded BASE URL: https://aimodels.jadeglobal.com:8082/ollama/api
Using MODEL: llama3.1:8b
Starting program...

Checking API connectivity...


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aimodels.jadeglobal.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


API is reachable.
Calling LLM API...


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aimodels.jadeglobal.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


❌ API ERROR: 405 Client Error: Method Not Allowed for url: https://aimodels.jadeglobal.com:8082/ollama/api/generate

Execution failed: 405 Client Error: Method Not Allowed for url: https://aimodels.jadeglobal.com:8082/ollama/api/generate

Program finished.
